In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup

from datetime import datetime
import networkx as nx
import requests
import pandas as pd

def get_df(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")

    df = pd.read_html(str(table))
    return df[0]


def df_to_travel_times(df, stops):
    df.columns = [d.split(" Stop ID")[0] for d in df.columns]
    col_times = dict(
        zip(df.columns, [time.strptime("0" + l[:4], "%H:%M") for l in df.iloc[0]])
    )
    return {k: datetime(*v[:5]) for k, v in col_times.items()}
time_dict = df_to_travel_times(get_df("https://trimet.org/schedules/w/t1017_0.htm"), None)

C:\Users\maxno\AppData\Local\Temp\ipykernel_31972\4175311602.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))


In [ ]:

def time_dict_to_digraph(stops):
    G = nx.DiGraph()

    # Create edges with time differences (in minutes)
    stop_names = list(stops.keys())

    for i in range(len(stop_names) - 1):
        start = stop_names[i]
        end = stop_names[i + 1]
        delta = (stops[end] - stops[start]).total_seconds() / 60  # in minutes
        G.add_edge(start, end, weight=delta)
    return G


G = time_dict_to_digraph(time_dict)

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
def get_stops_from_map(bus_number) -> list:
    # Optional: Set up Chrome options (e.g., headless mode)
    options = Options()
    # options.add_argument("--headless")  # Uncomment to run in headless mode

    # Path to your ChromeDriver (make sure it's in PATH or provide full path)
    driver = webdriver.Chrome(options=options)

    time.sleep(3)
    url = f"https://ride.trimet.org/home/route/{bus_number}/"
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    elements = soup.find_all(
        lambda tag: tag.has_attr("class")
        and any(cls.startswith("styled__StopListBody") for cls in tag["class"])
    )

    element = elements[0]
    buttons = element.find_all("button")

    stops = [button.text.strip() for button in buttons]
    return stops
stop_seq = get_stops_from_map(17)

In [11]:
stop_seq

['NE Dekum & 27th',
 'NE 27th & Highland',
 'NE 27th & Holman',
 'NE 27th & Ainsworth',
 'NE 27th & Jarrett',
 'NE 27th & Killingsworth',
 'NE 27th & Sumner',
 'NE 27th & Alberta',
 'NE 27th & Wygant',
 'NE 27th & Prescott',
 'NE Skidmore & 29th',
 'NE 29th & Bryce',
 'NE 29th & Alameda',
 'NE Regents & Ridgewood Dr',
 'NE 24th & Fremont',
 'NE 24th & Klickitat',
 'NE 24th & Siskiyou',
 'NE 24th & Knott',
 'NE 24th & Thompson',
 'NE 24th & Hancock',
 'NE Broadway & 24th',
 'NE Broadway & 21st',
 'NE Broadway & 16th',
 'NE Broadway & 12th',
 'NE Broadway & 9th',
 'NE Broadway & 7th',
 'NE Broadway & M L King',
 'NE Broadway & 2nd',
 'N Broadway & Vancouver',
 'N Broadway & Benton',
 'NW Irving & 6th',
 'NW 5th & Davis',
 'SW 5th & Harvey Milk',
 'SW 5th & Taylor',
 'SW 5th & Columbia',
 'SW 5th & Harrison',
 '300 Block SW Lincoln',
 'South Waterfront/S Moody',
 'OMSI/SE Water',
 'SE Division Pl & 8th',
 'SE Milwaukie & Gideon',
 'SE Powell & 12th',
 'SE 17th & Rhine',
 'SE 17th & Center

In [15]:
#CHAT GPT CODE

def insert_all_stops(G, stop_sequence):
    G_augmented = nx.DiGraph()

    i = 0
    while i < len(stop_sequence) - 1:
        current = stop_sequence[i]

        # Search for next known node in original graph
        j = i + 1
        while j < len(stop_sequence) and stop_sequence[j] not in G.nodes:
            j += 1

        if j >= len(stop_sequence):
            break

        next_known = stop_sequence[j]

        # Check if the original graph has an edge between current and next_known
        if G.has_edge(current, next_known):
            original_weight = G[current][next_known]['weight']
        else:
            # Try to look for a path through G
            try:
                original_weight = nx.shortest_path_length(G, current, next_known, weight='weight')
            except:
                # If there's no path, skip
                i = j
                continue

        # Get the full segment: current -> intermediate stops -> next_known
        segment = stop_sequence[i:j+1]
        num_edges = len(segment) - 1
        weight_per_edge = original_weight / num_edges

        # Add edges for each step
        for k in range(num_edges):
            G_augmented.add_edge(segment[k], segment[k + 1], weight=weight_per_edge)

        i = j
    return G_augmented

GA = insert_all_stops(G, stop_seq)
GA

In [34]:

def nx_to_if7(G):
    #TODO directionality
    BUS_NUMBER = 17 
    stops = [str(n) for n in G.nodes]
    stop1 = stops.pop(0)
    end_str = f"{stop1} is a room. "
    last_stop = stop1
    for s in stops:
        end_str += f"{s} is north of {last_stop}. "
        last_stop = s
    end_str += f"""\n\nThe {BUS_NUMBER} bus is a relatively-scheduled train-car in {stop1}. The waiting duration of the {BUS_NUMBER} bus is 1 minute. The t-schedule of the {BUS_NUMBER} bus is the Table of Bus Schedule.\n"""
    end_str += """
    Table of Bus Schedule
    transit time	destination
    """
    for n in G.edges(data=True):
        end_str += f"{round(n[2]['weight'])}\t{n[0]}\n"
    #have to get the last one
    n = [n for n in G.edges(data=True) ][-1]
    end_str += f"{round(n[2]['weight'])}\t{n[1]}\n"

    return end_str

nx_to_if7(G)

'NE Dekum & 27th is a room. NE 24th & Fremont is north of NE Dekum & 27th. NE Broadway & 12th is north of NE 24th & Fremont. NW Irving & 6th is north of NE Broadway & 12th. SW 5th & Taylor is north of NW Irving & 6th. South Waterfront/S Moody is north of SW 5th & Taylor. SE Powell & 12th is north of South Waterfront/S Moody. SE 17th & Center is north of SE Powell & 12th. SE Holgate & Cesar Chavez Blvd is north of SE 17th & Center. SE Holgate & 83rd is north of SE Holgate & Cesar Chavez Blvd. SE Holgate & 122nd is north of SE Holgate & 83rd. \n\nThe 17 bus is a relatively-scheduled train-car in NE Dekum & 27th. The waiting duration of the 17 bus is 1 minute. The t-schedule of the 17 bus is the Table of Bus Schedule.\n\n    Table of Bus Schedule\n    transit time\tdestination\n    9\tNE Dekum & 27th\n7\tNE 24th & Fremont\n7\tNE Broadway & 12th\n12\tNW Irving & 6th\n7\tSW 5th & Taylor\n6\tSouth Waterfront/S Moody\n2\tSE Powell & 12th\n6\tSE 17th & Center\n8\tSE Holgate & Cesar Chavez Blvd